ADRA ADSA UNTUK GA

In [4]:
import json
import itertools
import networkx as nx
import pandas as pd
import random
from typing import Union, List, Dict
from collections import defaultdict

# --- FUNGSI PEMUATAN DAN PERSIAPAN DATA ---

def load_json_file(filename: str) -> Union[dict, list]:
    """Membuka dan memuat data dari sebuah file JSON."""
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: File '{filename}' tidak ditemukan.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Gagal mem-parsing JSON dari file '{filename}'.")
        return None

def build_network_graph(network_data: dict) -> nx.Graph:
    """Membangun graf jaringan dari data networkDefinition.json."""
    G = nx.Graph()
    if not network_data: return G
    for entity in network_data.get('entity', []):
        G.add_node(entity['id'], **entity)
    for link in network_data.get('link', []):
        G.add_edge(link['s'], link['d'], PR=link['PR'], BW=link['BW'])
    return G

def build_placement_matrix(placement_data: dict) -> dict:
    """Membangun kamus penempatan dari data allocDefinition.json."""
    placement_matrix = defaultdict(list)
    if not placement_data: return {}
    for allocation in placement_data.get('initialAllocation', []):
        module_name = allocation["module_name"]
        resource_id = allocation["id_resource"]
        if resource_id not in placement_matrix[module_name]:
            placement_matrix[module_name].append(resource_id)
    return dict(placement_matrix)

# --- FUNGSI PERHITUNGAN DELAY (Sama seperti sebelumnya) ---

def _calculate_path_delay(path: list, graph: nx.Graph, packet_size: float) -> float:
    """Menghitung total delay di sepanjang sebuah path."""
    total_delay = 0
    for i in range(len(path) - 1):
        u, v = path[i], path[i+1]
        link_data = graph.get_edge_data(u, v)
        if link_data:
            bw_ms = link_data.get('BW', 1) / 1000.0
            pr_ms = link_data.get('PR', 0)
            transmission_delay = packet_size / bw_ms if bw_ms > 0 else float('inf')
            link_delay = pr_ms + transmission_delay
            total_delay += link_delay
    return total_delay

def calculate_adra(start_node: int, end_node: int, graph: nx.Graph, packet_size: float) -> float:
    """Menghitung ADRA: Delay dari requester ke titik masuk aplikasi."""
    try:
        path = nx.shortest_path(graph, source=start_node, target=end_node)
        return _calculate_path_delay(path, graph, packet_size)
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        return float('inf')

def calculate_adsa(target_app: dict, placement_matrix: dict, network_graph: nx.Graph, packet_size: float) -> float:
    """Menghitung ADSA: Rata-rata delay antar layanan dalam satu aplikasi."""
    service_list = [module['name'] for module in target_app.get('module', [])]
    all_device_locations = set()
    for service_name in service_list:
        if service_name in placement_matrix:
            all_device_locations.update(placement_matrix[service_name])
    if len(all_device_locations) < 2: return 0.0
    services_delay_list = []
    for dev1, dev2 in itertools.combinations(list(all_device_locations), 2):
        delay = calculate_adra(dev1, dev2, network_graph, packet_size)
        if delay != float('inf'):
            services_delay_list.append(delay)
    if not services_delay_list: return 0.0
    return sum(services_delay_list) / len(services_delay_list)

# --- BLOK EKSEKUSI UTAMA YANG DIMODIFIKASI ---
if __name__ == "__main__":
    # 1. Muat semua data dari file JSON (dengan path ke folder data)
    network_def = load_json_file('exp_rev/networkDefinition.json')
    app_def = load_json_file('exp_rev/appDefinition.json')
    alloc_def = load_json_file('exp_rev/allocDefinitionGA.json')

    if not all([network_def, app_def, alloc_def]):
        print("Satu atau lebih file definisi tidak dapat dimuat. Program berhenti.")
    else:
        # 2. Persiapkan struktur data
        network_graph = build_network_graph(network_def)
        placement_matrix = build_placement_matrix(alloc_def)
        
        # Siapkan daftar node untuk requester acak (kecuali node cloud ID 100)
        potential_requesters = [node['id'] for node in network_def.get('entity', []) if node.get('type') != 'CLOUD']
        if not potential_requesters:
            potential_requesters = [node for node in network_graph.nodes() if node != 100]

        # 3. Loop untuk semua aplikasi dan simpan hasilnya
        results_list = []
        PACKET_SIZE_BITS = 1_500_000  # Ukuran paket dalam bit (1.5 MB)

        for app_data in app_def:
            app_id = app_data['name']
            
            # Hitung ADSA
            adsa_result = calculate_adsa(app_data, placement_matrix, network_graph, PACKET_SIZE_BITS)
            
            # Hitung ADRA dengan requester acak
            entry_message = f"M.USER.APP.{app_id}"
            entry_module_name = next((t.get("module") for t in app_data.get("transmission", []) if t.get("message_in") == entry_message), None)
            
            adra_result = 0.0
            if entry_module_name and entry_module_name in placement_matrix and potential_requesters:
                entry_node = placement_matrix[entry_module_name][0]
                random_requester_id = random.choice(potential_requesters)
                adra_result = calculate_adra(random_requester_id, entry_node, network_graph, PACKET_SIZE_BITS)
            
            # Simpan hasil
            results_list.append({
                'app_id': app_id,
                'adra_ms': adra_result if adra_result != float('inf') else None,
                'adsa_ms': adsa_result,
                'total_delay_ms': (adra_result + adsa_result) if adra_result != float('inf') else None
            })

        # 4. Buat DataFrame dan lakukan analisis
        df_results = pd.DataFrame(results_list)
        df_results.set_index('app_id', inplace=True)

        # convert delay to seconds
        df_results['adra_s'] = df_results['adra_ms'] / 1000 if df_results['adra_ms'] is not None else None
        df_results['adsa_s'] = df_results['adsa_ms'] / 1000 if df_results['adsa_ms'] is not None else None
        df_results['total_delay_s'] = df_results['total_delay_ms'] / 1000 if df_results['total_delay_ms'] is not None else None
        
        print("--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---")
        print(df_results.to_string(float_format="%.2f"))
        print("\n" + "="*50 + "\n")

        print("--- Analisis Rata-rata Delay ---")
        
        # calculate average from adra_ms and adsa_ms
        average_adra = df_results['adra_ms'].mean()
        average_adra_s = average_adra / 1000 if average_adra is not None else None
        average_adsa = df_results['adsa_ms'].mean()
        average_adsa_s = average_adsa / 1000 if average_adsa is not None else None
        print(f"Rata-rata ADRA: {average_adra:.2f} ms ({average_adra_s:.2f} s)")
        print(f"Rata-rata ADSA: {average_adsa:.2f} ms ({average_adsa_s:.2f} s)")

        # Menghitung rata-rata keseluruhan dengan mengabaikan nilai None (inf)
        average_total_delay = df_results['total_delay_ms'].mean()
        print(f"Rata-rata Total Delay (ADRA + ADSA) untuk semua aplikasi: {average_total_delay:.2f} ms")
        
        # Aplikasi dengan delay terendah dan tertinggi
        app_min_delay = df_results['total_delay_ms'].idxmin()
        min_delay = df_results['total_delay_ms'].min()
        app_max_delay = df_results['total_delay_ms'].idxmax()
        max_delay = df_results['total_delay_ms'].max()
        
        print(f"Aplikasi dengan delay terendah: App '{app_min_delay}' ({min_delay:.2f} ms)")
        print(f"Aplikasi dengan delay tertinggi: App '{app_max_delay}' ({max_delay:.2f} ms)")


--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---
        adra_ms  adsa_ms  total_delay_ms  adra_s  adsa_s  total_delay_s
app_id                                                                 
0      80020.00 49344.67       129364.67   80.02   49.34         129.36
1      60015.00 58014.00       118029.00   60.02   58.01         118.03
2      60015.00 28005.67        88020.67   60.02   28.01          88.02
3      60015.00 60871.86       120886.86   60.02   60.87         120.89
4      40010.00 46010.50        86020.50   40.01   46.01          86.02
5      60015.00 56013.33       116028.33   60.02   56.01         116.03
6      40010.00 52012.43        92022.43   40.01   52.01          92.02
7      20005.00 57124.94        77129.94   20.00   57.12          77.13
8          0.00 54679.67        54679.67    0.00   54.68          54.68
9      60015.00 60814.40       120829.40   60.02   60.81         120.83
10     60015.00 49900.92       109915.92   60.02   49.90         109.92
11     400

In [5]:
if __name__ == "__main__":
    # 1. Muat semua data dari file JSON
    network_def = load_json_file('exp_rev/networkDefinition.json')
    app_def = load_json_file('exp_rev/appDefinition.json')
    alloc_def = load_json_file('exp_rev/allocDefinitionGA.json')

    if not all([network_def, app_def, alloc_def]):
        print("Satu atau lebih file definisi tidak dapat dimuat. Program berhenti.")
    else:
        # 2. Persiapkan struktur data
        network_graph = build_network_graph(network_def)
        placement_matrix = build_placement_matrix(alloc_def)
        
        # Siapkan daftar node untuk requester acak (kecuali node cloud ID 100)
        potential_requesters = [node['id'] for node in network_def.get('entity', []) if node.get('type') != 'CLOUD']
        if not potential_requesters:
            potential_requesters = [node for node in network_graph.nodes() if node != 100]

        # 3. Loop untuk semua aplikasi dan simpan hasilnya
        results_list = []
        PACKET_SIZE_BITS = 1_500_000  # Ukuran paket dalam bit (1.5 MB)

        for app_data in app_def:
            app_id = app_data['name']
            
            # Hitung ADSA
            adsa_result = calculate_adsa(app_data, placement_matrix, network_graph, PACKET_SIZE_BITS)
            
            # Hitung ADRA dengan requester acak
            entry_message = f"M.USER.APP.{app_id}"
            entry_module_name = next((t.get("module") for t in app_data.get("transmission", []) if t.get("message_in") == entry_message), None)
            
            adra_result = 0.0
            if entry_module_name and entry_module_name in placement_matrix and potential_requesters:
                entry_node = placement_matrix[entry_module_name][0]
                random_requester_id = random.choice(potential_requesters)
                adra_result = calculate_adra(random_requester_id, entry_node, network_graph, PACKET_SIZE_BITS)
            
            # Simpan hasil
            results_list.append({
                'app_id': app_id,
                'adra_ms': adra_result if adra_result != float('inf') else None,
                'adsa_ms': adsa_result,
                'total_delay_ms': (adra_result + adsa_result) if adra_result != float('inf') else None
            })

        # 4. Buat DataFrame dan lakukan analisis
        df_results = pd.DataFrame(results_list)
        df_results.set_index('app_id', inplace=True)

        # convert delay to seconds
        df_results['adra_s'] = df_results['adra_ms'] / 1000 if df_results['adra_ms'] is not None else None
        df_results['adsa_s'] = df_results['adsa_ms'] / 1000 if df_results['adsa_ms'] is not None else None
        df_results['total_delay_s'] = df_results['total_delay_ms'] / 1000 if df_results['total_delay_ms'] is not None else None
        
        print("--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---")
        print(df_results.to_string(float_format="%.2f"))
        print("\n" + "="*50 + "\n")

        print("--- Analisis Rata-rata Delay ---")
        
        # calculate average from adra_ms and adsa_ms
        average_adra = df_results['adra_ms'].mean()
        average_adra_s = average_adra / 1000 if average_adra is not None else None
        average_adsa = df_results['adsa_ms'].mean()
        average_adsa_s = average_adsa / 1000 if average_adsa is not None else None
        print(f"Rata-rata ADRA: {average_adra:.2f} ms ({average_adra_s:.2f} s)")
        print(f"Rata-rata ADSA: {average_adsa:.2f} ms ({average_adsa_s:.2f} s)")

        # Menghitung rata-rata keseluruhan dengan mengabaikan nilai None (inf)
        average_total_delay = df_results['total_delay_ms'].mean()
        print(f"Rata-rata Total Delay (ADRA + ADSA) untuk semua aplikasi: {average_total_delay:.2f} ms")
        
        # Aplikasi dengan delay terendah dan tertinggi
        app_min_delay = df_results['total_delay_ms'].idxmin()
        min_delay = df_results['total_delay_ms'].min()
        app_max_delay = df_results['total_delay_ms'].idxmax()
        max_delay = df_results['total_delay_ms'].max()
        
        print(f"Aplikasi dengan delay terendah: App '{app_min_delay}' ({min_delay:.2f} ms)")
        print(f"Aplikasi dengan delay tertinggi: App '{app_max_delay}' ({max_delay:.2f} ms)")

--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---
        adra_ms  adsa_ms  total_delay_ms  adra_s  adsa_s  total_delay_s
app_id                                                                 
0      80020.00 49344.67       129364.67   80.02   49.34         129.36
1      40010.00 58014.00        98024.00   40.01   58.01          98.02
2      40010.00 28005.67        68015.67   40.01   28.01          68.02
3      60015.00 60871.86       120886.86   60.02   60.87         120.89
4      40010.00 46010.50        86020.50   40.01   46.01          86.02
5      40010.00 56013.33        96023.33   40.01   56.01          96.02
6      40010.00 52012.43        92022.43   40.01   52.01          92.02
7      60015.00 57124.94       117139.94   60.02   57.12         117.14
8      40010.00 54679.67        94689.67   40.01   54.68          94.69
9      60015.00 60814.40       120829.40   60.02   60.81         120.83
10     60015.00 49900.92       109915.92   60.02   49.90         109.92
11     200

In [6]:
# based on appDefinition.json, allocDefinition.json, networkDefinition.json
# give me list of client id and entry point for each application

def load_app_entry_points(app_def: list, placement_matrix: dict) -> dict:
    """Mengambil daftar ID klien dan titik masuk untuk setiap aplikasi."""
    app_entry_points = {}
    for app in app_def:
        app_id = app['name']
        entry_message = f"M.USER.APP.{app_id}"
        entry_module_name = next((t.get("module") for t in app.get("transmission", []) if t.get("message_in") == entry_message), None)
        
        if entry_module_name and entry_module_name in placement_matrix:
            # Get all entry points for this module
            entry_nodes = placement_matrix[entry_module_name]
            app_entry_points[app_id] = entry_nodes
    return app_entry_points

# Get entry points for all applications
app_entry_points = load_app_entry_points(app_def, placement_matrix)

# Print in the requested format
for app_id, entry_points_list in app_entry_points.items():
    print(f"app: {app_id}, entry_point: {entry_points_list}")

# Create a summary DataFrame for better visualization
entry_points_summary = []
for app_id, entry_points_list in app_entry_points.items():
    entry_points_summary.append({
        'app_id': app_id,
        'entry_points': entry_points_list,
        'num_entry_points': len(entry_points_list)
    })

summary_df = pd.DataFrame(entry_points_summary)
summary_df.set_index('app_id', inplace=True)

print("\n--- Summary Table: Application Entry Points ---")
print(summary_df.to_string())

# Show statistics
print(f"\nTotal applications: {len(app_entry_points)}")
all_entry_points = [ep for ep_list in app_entry_points.values() for ep in ep_list]
print(f"Total entry points across all apps: {len(all_entry_points)}")
print(f"Unique entry points: {len(set(all_entry_points))}")

app: 0, entry_point: [9, 100]
app: 1, entry_point: [13, 100]
app: 2, entry_point: [81, 100]
app: 3, entry_point: [39, 100]
app: 4, entry_point: [2, 100]
app: 5, entry_point: [3, 100]
app: 6, entry_point: [6, 100]
app: 7, entry_point: [2, 100]
app: 8, entry_point: [59, 100]
app: 9, entry_point: [61, 100]
app: 10, entry_point: [7, 100]
app: 11, entry_point: [2, 100]
app: 12, entry_point: [1, 100]
app: 13, entry_point: [6, 100]
app: 14, entry_point: [2, 100]
app: 15, entry_point: [6, 100]
app: 16, entry_point: [98, 100]
app: 17, entry_point: [1, 100]
app: 18, entry_point: [89, 100]
app: 19, entry_point: [0, 100]

--- Summary Table: Application Entry Points ---
       entry_points  num_entry_points
app_id                               
0          [9, 100]                 2
1         [13, 100]                 2
2         [81, 100]                 2
3         [39, 100]                 2
4          [2, 100]                 2
5          [3, 100]                 2
6          [6, 100]           

In [7]:
# Get unique entry point nodes across all applications
unique_entry_points = set()
for entry_points_list in app_entry_points.values():
    unique_entry_points.update(entry_points_list)

# Convert to sorted list for better readability
unique_entry_points_sorted = sorted(list(unique_entry_points))

print("--- Recap: Unique Entry Point Nodes ---")
print(f"Unique entry point nodes: {unique_entry_points_sorted}")
print(f"Total unique entry point nodes: {len(unique_entry_points_sorted)}")

# Create a frequency analysis to see which nodes are most commonly used as entry points
entry_point_frequency = {}
for entry_points_list in app_entry_points.values():
    for node in entry_points_list:
        entry_point_frequency[node] = entry_point_frequency.get(node, 0) + 1

# Sort by frequency (descending)
sorted_by_frequency = sorted(entry_point_frequency.items(), key=lambda x: x[1], reverse=True)

print("\n--- Entry Point Node Frequency Analysis ---")
print("Node ID | Frequency (Number of apps using this node)")
print("-" * 50)
for node_id, freq in sorted_by_frequency:
    print(f"{node_id:7} | {freq}")

# Identify cloud nodes vs edge nodes
cloud_nodes = [node for node in unique_entry_points_sorted if node == 100]  # Assuming 100 is cloud
edge_nodes = [node for node in unique_entry_points_sorted if node != 100]

print("\n--- Node Type Classification ---")
print(f"Cloud nodes serving as entry points: {cloud_nodes}")
print(f"Edge nodes serving as entry points: {edge_nodes}")
print(f"Number of edge nodes: {len(edge_nodes)}")
print(f"Number of cloud nodes: {len(cloud_nodes)}")

--- Recap: Unique Entry Point Nodes ---
Unique entry point nodes: [0, 1, 2, 3, 6, 7, 9, 13, 39, 59, 61, 81, 89, 98, 100]
Total unique entry point nodes: 15

--- Entry Point Node Frequency Analysis ---
Node ID | Frequency (Number of apps using this node)
--------------------------------------------------
    100 | 20
      2 | 4
      6 | 3
      1 | 2
      9 | 1
     13 | 1
     81 | 1
     39 | 1
      3 | 1
     59 | 1
     61 | 1
      7 | 1
     98 | 1
     89 | 1
      0 | 1

--- Node Type Classification ---
Cloud nodes serving as entry points: [100]
Edge nodes serving as entry points: [0, 1, 2, 3, 6, 7, 9, 13, 39, 59, 61, 81, 89, 98]
Number of edge nodes: 14
Number of cloud nodes: 1


DELAY ADSA ADRA UNTUK ILP

In [1]:
import json
import itertools
import networkx as nx
import pandas as pd
import random
from typing import Union, List, Dict
from collections import defaultdict

# --- FUNGSI PEMUATAN DAN PERSIAPAN DATA ---

def load_json_file(filename: str) -> Union[dict, list]:
    """Membuka dan memuat data dari sebuah file JSON."""
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: File '{filename}' tidak ditemukan.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Gagal mem-parsing JSON dari file '{filename}'.")
        return None

def build_network_graph(network_data: dict) -> nx.Graph:
    """Membangun graf jaringan dari data networkDefinition.json."""
    G = nx.Graph()
    if not network_data: return G
    for entity in network_data.get('entity', []):
        G.add_node(entity['id'], **entity)
    for link in network_data.get('link', []):
        G.add_edge(link['s'], link['d'], PR=link['PR'], BW=link['BW'])
    return G

def build_placement_matrix(placement_data: dict) -> dict:
    """Membangun kamus penempatan dari data allocDefinition.json."""
    placement_matrix = defaultdict(list)
    if not placement_data: return {}
    for allocation in placement_data.get('initialAllocation', []):
        module_name = allocation["module_name"]
        resource_id = allocation["id_resource"]
        if resource_id not in placement_matrix[module_name]:
            placement_matrix[module_name].append(resource_id)
    return dict(placement_matrix)

# --- FUNGSI PERHITUNGAN DELAY (Sama seperti sebelumnya) ---

def _calculate_path_delay(path: list, graph: nx.Graph, packet_size: float) -> float:
    """Menghitung total delay di sepanjang sebuah path."""
    total_delay = 0
    for i in range(len(path) - 1):
        u, v = path[i], path[i+1]
        link_data = graph.get_edge_data(u, v)
        if link_data:
            bw_ms = link_data.get('BW', 1) / 1000.0
            pr_ms = link_data.get('PR', 0)
            transmission_delay = packet_size / bw_ms if bw_ms > 0 else float('inf')
            link_delay = pr_ms + transmission_delay
            total_delay += link_delay
    return total_delay

def calculate_adra(start_node: int, end_node: int, graph: nx.Graph, packet_size: float) -> float:
    """Menghitung ADRA: Delay dari requester ke titik masuk aplikasi."""
    try:
        path = nx.shortest_path(graph, source=start_node, target=end_node)
        return _calculate_path_delay(path, graph, packet_size)
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        return float('inf')

def calculate_adsa(target_app: dict, placement_matrix: dict, network_graph: nx.Graph, packet_size: float) -> float:
    """Menghitung ADSA: Rata-rata delay antar layanan dalam satu aplikasi."""
    service_list = [module['name'] for module in target_app.get('module', [])]
    all_device_locations = set()
    for service_name in service_list:
        if service_name in placement_matrix:
            all_device_locations.update(placement_matrix[service_name])
    if len(all_device_locations) < 2: return 0.0
    services_delay_list = []
    for dev1, dev2 in itertools.combinations(list(all_device_locations), 2):
        delay = calculate_adra(dev1, dev2, network_graph, packet_size)
        if delay != float('inf'):
            services_delay_list.append(delay)
    if not services_delay_list: return 0.0
    return sum(services_delay_list) / len(services_delay_list)

# --- BLOK EKSEKUSI UTAMA YANG DIMODIFIKASI ---
if __name__ == "__main__":
    # 1. Muat semua data dari file JSON (dengan path ke folder data)
    network_def = load_json_file('exp_rev/networkDefinition.json')
    app_def = load_json_file('exp_rev/appDefinition.json')
    alloc_def = load_json_file('exp_rev/allocDefinitionilp.json')

    if not all([network_def, app_def, alloc_def]):
        print("Satu atau lebih file definisi tidak dapat dimuat. Program berhenti.")
    else:
        # 2. Persiapkan struktur data
        network_graph = build_network_graph(network_def)
        placement_matrix = build_placement_matrix(alloc_def)
        
        # Siapkan daftar node untuk requester acak (kecuali node cloud ID 100)
        potential_requesters = [node['id'] for node in network_def.get('entity', []) if node.get('type') != 'CLOUD']
        if not potential_requesters:
            potential_requesters = [node for node in network_graph.nodes() if node != 100]

        # 3. Loop untuk semua aplikasi dan simpan hasilnya
        results_list = []
        PACKET_SIZE_BITS = 1_500_000  # Ukuran paket dalam bit (1.5 MB)

        for app_data in app_def:
            app_id = app_data['name']
            
            # Hitung ADSA
            adsa_result = calculate_adsa(app_data, placement_matrix, network_graph, PACKET_SIZE_BITS)
            
            # Hitung ADRA dengan requester acak
            entry_message = f"M.USER.APP.{app_id}"
            entry_module_name = next((t.get("module") for t in app_data.get("transmission", []) if t.get("message_in") == entry_message), None)
            
            adra_result = 0.0
            if entry_module_name and entry_module_name in placement_matrix and potential_requesters:
                entry_node = placement_matrix[entry_module_name][0]
                random_requester_id = random.choice(potential_requesters)
                adra_result = calculate_adra(random_requester_id, entry_node, network_graph, PACKET_SIZE_BITS)
            
            # Simpan hasil
            results_list.append({
                'app_id': app_id,
                'adra_ms': adra_result if adra_result != float('inf') else None,
                'adsa_ms': adsa_result,
                'total_delay_ms': (adra_result + adsa_result) if adra_result != float('inf') else None
            })

        # 4. Buat DataFrame dan lakukan analisis
        df_results = pd.DataFrame(results_list)
        df_results.set_index('app_id', inplace=True)

        # convert delay to seconds
        df_results['adra_s'] = df_results['adra_ms'] / 1000 if df_results['adra_ms'] is not None else None
        df_results['adsa_s'] = df_results['adsa_ms'] / 1000 if df_results['adsa_ms'] is not None else None
        df_results['total_delay_s'] = df_results['total_delay_ms'] / 1000 if df_results['total_delay_ms'] is not None else None
        
        print("--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---")
        print(df_results.to_string(float_format="%.2f"))
        print("\n" + "="*50 + "\n")

        print("--- Analisis Rata-rata Delay ---")
        
        # calculate average from adra_ms and adsa_ms
        average_adra = df_results['adra_ms'].mean()
        average_adra_s = average_adra / 1000 if average_adra is not None else None
        average_adsa = df_results['adsa_ms'].mean()
        average_adsa_s = average_adsa / 1000 if average_adsa is not None else None
        print(f"Rata-rata ADRA: {average_adra:.2f} ms ({average_adra_s:.2f} s)")
        print(f"Rata-rata ADSA: {average_adsa:.2f} ms ({average_adsa_s:.2f} s)")

        # Menghitung rata-rata keseluruhan dengan mengabaikan nilai None (inf)
        average_total_delay = df_results['total_delay_ms'].mean()
        print(f"Rata-rata Total Delay (ADRA + ADSA) untuk semua aplikasi: {average_total_delay:.2f} ms")
        
        # Aplikasi dengan delay terendah dan tertinggi
        app_min_delay = df_results['total_delay_ms'].idxmin()
        min_delay = df_results['total_delay_ms'].min()
        app_max_delay = df_results['total_delay_ms'].idxmax()
        max_delay = df_results['total_delay_ms'].max()
        
        print(f"Aplikasi dengan delay terendah: App '{app_min_delay}' ({min_delay:.2f} ms)")
        print(f"Aplikasi dengan delay tertinggi: App '{app_max_delay}' ({max_delay:.2f} ms)")


--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---
        adra_ms  adsa_ms  total_delay_ms  adra_s  adsa_s  total_delay_s
app_id                                                                 
0      60015.00 48010.67       108025.67   60.02   48.01         108.03
1      40010.00 61348.44       101358.44   40.01   61.35         101.36
2      60015.00 48010.67       108025.67   60.02   48.01         108.03
3      80020.00 36008.00       116028.00   80.02   36.01         116.03
4      60015.00 49154.57       109169.57   60.02   49.15         109.17
5      80020.00 41342.33       121362.33   80.02   41.34         121.36
6      60015.00 49440.43       109455.43   60.02   49.44         109.46
7      60015.00 47746.99       107761.99   60.02   47.75         107.76
8      40010.00 42295.79        82305.79   40.01   42.30          82.31
9      80020.00 46810.90       126830.90   80.02   46.81         126.83
10     60015.00 52921.98       112936.98   60.02   52.92         112.94
11     800

In [2]:
if __name__ == "__main__":
    # 1. Muat semua data dari file JSON
    network_def = load_json_file('exp_rev/networkDefinition.json')
    app_def = load_json_file('exp_rev/appDefinition.json')
    alloc_def = load_json_file('exp_rev/allocDefinitionILP.json')

    if not all([network_def, app_def, alloc_def]):
        print("Satu atau lebih file definisi tidak dapat dimuat. Program berhenti.")
    else:
        # 2. Persiapkan struktur data
        network_graph = build_network_graph(network_def)
        placement_matrix = build_placement_matrix(alloc_def)
        
        # Siapkan daftar node untuk requester acak (kecuali node cloud ID 100)
        potential_requesters = [node['id'] for node in network_def.get('entity', []) if node.get('type') != 'CLOUD']
        if not potential_requesters:
            potential_requesters = [node for node in network_graph.nodes() if node != 100]

        # 3. Loop untuk semua aplikasi dan simpan hasilnya
        results_list = []
        PACKET_SIZE_BITS = 1_500_000  # Ukuran paket dalam bit (1.5 MB)

        for app_data in app_def:
            app_id = app_data['name']
            
            # Hitung ADSA
            adsa_result = calculate_adsa(app_data, placement_matrix, network_graph, PACKET_SIZE_BITS)
            
            # Hitung ADRA dengan requester acak
            entry_message = f"M.USER.APP.{app_id}"
            entry_module_name = next((t.get("module") for t in app_data.get("transmission", []) if t.get("message_in") == entry_message), None)
            
            adra_result = 0.0
            if entry_module_name and entry_module_name in placement_matrix and potential_requesters:
                entry_node = placement_matrix[entry_module_name][0]
                random_requester_id = random.choice(potential_requesters)
                adra_result = calculate_adra(random_requester_id, entry_node, network_graph, PACKET_SIZE_BITS)
            
            # Simpan hasil
            results_list.append({
                'app_id': app_id,
                'adra_ms': adra_result if adra_result != float('inf') else None,
                'adsa_ms': adsa_result,
                'total_delay_ms': (adra_result + adsa_result) if adra_result != float('inf') else None
            })

        # 4. Buat DataFrame dan lakukan analisis
        df_results = pd.DataFrame(results_list)
        df_results.set_index('app_id', inplace=True)

        # convert delay to seconds
        df_results['adra_s'] = df_results['adra_ms'] / 1000 if df_results['adra_ms'] is not None else None
        df_results['adsa_s'] = df_results['adsa_ms'] / 1000 if df_results['adsa_ms'] is not None else None
        df_results['total_delay_s'] = df_results['total_delay_ms'] / 1000 if df_results['total_delay_ms'] is not None else None
        
        print("--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---")
        print(df_results.to_string(float_format="%.2f"))
        print("\n" + "="*50 + "\n")

        print("--- Analisis Rata-rata Delay ---")
        
        # calculate average from adra_ms and adsa_ms
        average_adra = df_results['adra_ms'].mean()
        average_adra_s = average_adra / 1000 if average_adra is not None else None
        average_adsa = df_results['adsa_ms'].mean()
        average_adsa_s = average_adsa / 1000 if average_adsa is not None else None
        print(f"Rata-rata ADRA: {average_adra:.2f} ms ({average_adra_s:.2f} s)")
        print(f"Rata-rata ADSA: {average_adsa:.2f} ms ({average_adsa_s:.2f} s)")

        # Menghitung rata-rata keseluruhan dengan mengabaikan nilai None (inf)
        average_total_delay = df_results['total_delay_ms'].mean()
        print(f"Rata-rata Total Delay (ADRA + ADSA) untuk semua aplikasi: {average_total_delay:.2f} ms")
        
        # Aplikasi dengan delay terendah dan tertinggi
        app_min_delay = df_results['total_delay_ms'].idxmin()
        min_delay = df_results['total_delay_ms'].min()
        app_max_delay = df_results['total_delay_ms'].idxmax()
        max_delay = df_results['total_delay_ms'].max()
        
        print(f"Aplikasi dengan delay terendah: App '{app_min_delay}' ({min_delay:.2f} ms)")
        print(f"Aplikasi dengan delay tertinggi: App '{app_max_delay}' ({max_delay:.2f} ms)")

--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---
        adra_ms  adsa_ms  total_delay_ms  adra_s  adsa_s  total_delay_s
app_id                                                                 
0      60015.00 48010.67       108025.67   60.02   48.01         108.03
1      60015.00 61348.44       121363.44   60.02   61.35         121.36
2      60015.00 48010.67       108025.67   60.02   48.01         108.03
3      40010.00 36008.00        76018.00   40.01   36.01          76.02
4      60015.00 49154.57       109169.57   60.02   49.15         109.17
5      80020.00 41342.33       121362.33   80.02   41.34         121.36
6      60015.00 49440.43       109455.43   60.02   49.44         109.46
7      40010.00 47746.99        87756.99   40.01   47.75          87.76
8      80020.00 42295.79       122315.79   80.02   42.30         122.32
9      80020.00 46810.90       126830.90   80.02   46.81         126.83
10     20005.00 52921.98        72926.98   20.00   52.92          72.93
11     600

In [3]:
# based on appDefinition.json, allocDefinition.json, networkDefinition.json
# give me list of client id and entry point for each application

def load_app_entry_points(app_def: list, placement_matrix: dict) -> dict:
    """Mengambil daftar ID klien dan titik masuk untuk setiap aplikasi."""
    app_entry_points = {}
    for app in app_def:
        app_id = app['name']
        entry_message = f"M.USER.APP.{app_id}"
        entry_module_name = next((t.get("module") for t in app.get("transmission", []) if t.get("message_in") == entry_message), None)
        
        if entry_module_name and entry_module_name in placement_matrix:
            # Get all entry points for this module
            entry_nodes = placement_matrix[entry_module_name]
            app_entry_points[app_id] = entry_nodes
    return app_entry_points

# Get entry points for all applications
app_entry_points = load_app_entry_points(app_def, placement_matrix)

# Print in the requested format
for app_id, entry_points_list in app_entry_points.items():
    print(f"app: {app_id}, entry_point: {entry_points_list}")

# Create a summary DataFrame for better visualization
entry_points_summary = []
for app_id, entry_points_list in app_entry_points.items():
    entry_points_summary.append({
        'app_id': app_id,
        'entry_points': entry_points_list,
        'num_entry_points': len(entry_points_list)
    })

summary_df = pd.DataFrame(entry_points_summary)
summary_df.set_index('app_id', inplace=True)

print("\n--- Summary Table: Application Entry Points ---")
print(summary_df.to_string())

# Show statistics
print(f"\nTotal applications: {len(app_entry_points)}")
all_entry_points = [ep for ep_list in app_entry_points.values() for ep in ep_list]
print(f"Total entry points across all apps: {len(all_entry_points)}")
print(f"Unique entry points: {len(set(all_entry_points))}")

app: 0, entry_point: [72, 100]
app: 1, entry_point: [23, 50, 34, 41, 100]
app: 2, entry_point: [56, 100]
app: 3, entry_point: [25, 100]
app: 4, entry_point: [65, 98, 97, 71, 14, 55, 100]
app: 5, entry_point: [10, 100]
app: 6, entry_point: [32, 98, 83, 62, 46, 100]
app: 7, entry_point: [9, 43, 3, 1, 21, 100]
app: 8, entry_point: [24, 46, 100]
app: 9, entry_point: [31, 100]
app: 10, entry_point: [39, 26, 100]
app: 11, entry_point: [28, 96, 67, 6, 100]
app: 12, entry_point: [4, 19, 58, 13, 21, 100]
app: 13, entry_point: [56, 67, 28, 89, 1, 0, 100]
app: 14, entry_point: [64, 96, 98, 97, 77, 81, 83, 52, 87, 100]
app: 15, entry_point: [20, 47, 44, 24, 93, 6, 100]
app: 16, entry_point: [88, 65, 81, 61, 100]
app: 17, entry_point: [15, 51, 60, 100]
app: 18, entry_point: [52, 61, 100]
app: 19, entry_point: [7, 100]

--- Summary Table: Application Entry Points ---
                                     entry_points  num_entry_points
app_id                                                            

In [4]:
# Get unique entry point nodes across all applications
unique_entry_points = set()
for entry_points_list in app_entry_points.values():
    unique_entry_points.update(entry_points_list)

# Convert to sorted list for better readability
unique_entry_points_sorted = sorted(list(unique_entry_points))

print("--- Recap: Unique Entry Point Nodes ---")
print(f"Unique entry point nodes: {unique_entry_points_sorted}")
print(f"Total unique entry point nodes: {len(unique_entry_points_sorted)}")

# Create a frequency analysis to see which nodes are most commonly used as entry points
entry_point_frequency = {}
for entry_points_list in app_entry_points.values():
    for node in entry_points_list:
        entry_point_frequency[node] = entry_point_frequency.get(node, 0) + 1

# Sort by frequency (descending)
sorted_by_frequency = sorted(entry_point_frequency.items(), key=lambda x: x[1], reverse=True)

print("\n--- Entry Point Node Frequency Analysis ---")
print("Node ID | Frequency (Number of apps using this node)")
print("-" * 50)
for node_id, freq in sorted_by_frequency:
    print(f"{node_id:7} | {freq}")

# Identify cloud nodes vs edge nodes
cloud_nodes = [node for node in unique_entry_points_sorted if node == 100]  # Assuming 100 is cloud
edge_nodes = [node for node in unique_entry_points_sorted if node != 100]

print("\n--- Node Type Classification ---")
print(f"Cloud nodes serving as entry points: {cloud_nodes}")
print(f"Edge nodes serving as entry points: {edge_nodes}")
print(f"Number of edge nodes: {len(edge_nodes)}")
print(f"Number of cloud nodes: {len(cloud_nodes)}")

--- Recap: Unique Entry Point Nodes ---
Unique entry point nodes: [0, 1, 3, 4, 6, 7, 9, 10, 13, 14, 15, 19, 20, 21, 23, 24, 25, 26, 28, 31, 32, 34, 39, 41, 43, 44, 46, 47, 50, 51, 52, 55, 56, 58, 60, 61, 62, 64, 65, 67, 71, 72, 77, 81, 83, 87, 88, 89, 93, 96, 97, 98, 100]
Total unique entry point nodes: 53

--- Entry Point Node Frequency Analysis ---
Node ID | Frequency (Number of apps using this node)
--------------------------------------------------
    100 | 20
     98 | 3
     56 | 2
     65 | 2
     97 | 2
     83 | 2
     46 | 2
      1 | 2
     21 | 2
     24 | 2
     28 | 2
     96 | 2
     67 | 2
      6 | 2
     81 | 2
     52 | 2
     61 | 2
     72 | 1
     23 | 1
     50 | 1
     34 | 1
     41 | 1
     25 | 1
     71 | 1
     14 | 1
     55 | 1
     10 | 1
     32 | 1
     62 | 1
      9 | 1
     43 | 1
      3 | 1
     31 | 1
     39 | 1
     26 | 1
      4 | 1
     19 | 1
     58 | 1
     13 | 1
     89 | 1
      0 | 1
     64 | 1
     77 | 1
     87 | 1
     20 | 1
  

DELAY ADSA ADRA UNTUK PARTITION

In [5]:
import json
import itertools
import networkx as nx
import pandas as pd
import random
from typing import Union, List, Dict
from collections import defaultdict

# --- FUNGSI PEMUATAN DAN PERSIAPAN DATA ---

def load_json_file(filename: str) -> Union[dict, list]:
    """Membuka dan memuat data dari sebuah file JSON."""
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: File '{filename}' tidak ditemukan.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Gagal mem-parsing JSON dari file '{filename}'.")
        return None

def build_network_graph(network_data: dict) -> nx.Graph:
    """Membangun graf jaringan dari data networkDefinition.json."""
    G = nx.Graph()
    if not network_data: return G
    for entity in network_data.get('entity', []):
        G.add_node(entity['id'], **entity)
    for link in network_data.get('link', []):
        G.add_edge(link['s'], link['d'], PR=link['PR'], BW=link['BW'])
    return G

def build_placement_matrix(placement_data: dict) -> dict:
    """Membangun kamus penempatan dari data allocDefinition.json."""
    placement_matrix = defaultdict(list)
    if not placement_data: return {}
    for allocation in placement_data.get('initialAllocation', []):
        module_name = allocation["module_name"]
        resource_id = allocation["id_resource"]
        if resource_id not in placement_matrix[module_name]:
            placement_matrix[module_name].append(resource_id)
    return dict(placement_matrix)

# --- FUNGSI PERHITUNGAN DELAY (Sama seperti sebelumnya) ---

def _calculate_path_delay(path: list, graph: nx.Graph, packet_size: float) -> float:
    """Menghitung total delay di sepanjang sebuah path."""
    total_delay = 0
    for i in range(len(path) - 1):
        u, v = path[i], path[i+1]
        link_data = graph.get_edge_data(u, v)
        if link_data:
            bw_ms = link_data.get('BW', 1) / 1000.0
            pr_ms = link_data.get('PR', 0)
            transmission_delay = packet_size / bw_ms if bw_ms > 0 else float('inf')
            link_delay = pr_ms + transmission_delay
            total_delay += link_delay
    return total_delay

def calculate_adra(start_node: int, end_node: int, graph: nx.Graph, packet_size: float) -> float:
    """Menghitung ADRA: Delay dari requester ke titik masuk aplikasi."""
    try:
        path = nx.shortest_path(graph, source=start_node, target=end_node)
        return _calculate_path_delay(path, graph, packet_size)
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        return float('inf')

def calculate_adsa(target_app: dict, placement_matrix: dict, network_graph: nx.Graph, packet_size: float) -> float:
    """Menghitung ADSA: Rata-rata delay antar layanan dalam satu aplikasi."""
    service_list = [module['name'] for module in target_app.get('module', [])]
    all_device_locations = set()
    for service_name in service_list:
        if service_name in placement_matrix:
            all_device_locations.update(placement_matrix[service_name])
    if len(all_device_locations) < 2: return 0.0
    services_delay_list = []
    for dev1, dev2 in itertools.combinations(list(all_device_locations), 2):
        delay = calculate_adra(dev1, dev2, network_graph, packet_size)
        if delay != float('inf'):
            services_delay_list.append(delay)
    if not services_delay_list: return 0.0
    return sum(services_delay_list) / len(services_delay_list)

# --- BLOK EKSEKUSI UTAMA YANG DIMODIFIKASI ---
if __name__ == "__main__":
    # 1. Muat semua data dari file JSON (dengan path ke folder data)
    network_def = load_json_file('exp_rev/networkDefinition.json')
    app_def = load_json_file('exp_rev/appDefinition.json')
    alloc_def = load_json_file('exp_rev/allocDefinition.json')

    if not all([network_def, app_def, alloc_def]):
        print("Satu atau lebih file definisi tidak dapat dimuat. Program berhenti.")
    else:
        # 2. Persiapkan struktur data
        network_graph = build_network_graph(network_def)
        placement_matrix = build_placement_matrix(alloc_def)
        
        # Siapkan daftar node untuk requester acak (kecuali node cloud ID 100)
        potential_requesters = [node['id'] for node in network_def.get('entity', []) if node.get('type') != 'CLOUD']
        if not potential_requesters:
            potential_requesters = [node for node in network_graph.nodes() if node != 100]

        # 3. Loop untuk semua aplikasi dan simpan hasilnya
        results_list = []
        PACKET_SIZE_BITS = 1_500_000  # Ukuran paket dalam bit (1.5 MB)

        for app_data in app_def:
            app_id = app_data['name']
            
            # Hitung ADSA
            adsa_result = calculate_adsa(app_data, placement_matrix, network_graph, PACKET_SIZE_BITS)
            
            # Hitung ADRA dengan requester acak
            entry_message = f"M.USER.APP.{app_id}"
            entry_module_name = next((t.get("module") for t in app_data.get("transmission", []) if t.get("message_in") == entry_message), None)
            
            adra_result = 0.0
            if entry_module_name and entry_module_name in placement_matrix and potential_requesters:
                entry_node = placement_matrix[entry_module_name][0]
                random_requester_id = random.choice(potential_requesters)
                adra_result = calculate_adra(random_requester_id, entry_node, network_graph, PACKET_SIZE_BITS)
            
            # Simpan hasil
            results_list.append({
                'app_id': app_id,
                'adra_ms': adra_result if adra_result != float('inf') else None,
                'adsa_ms': adsa_result,
                'total_delay_ms': (adra_result + adsa_result) if adra_result != float('inf') else None
            })

        # 4. Buat DataFrame dan lakukan analisis
        df_results = pd.DataFrame(results_list)
        df_results.set_index('app_id', inplace=True)

        # convert delay to seconds
        df_results['adra_s'] = df_results['adra_ms'] / 1000 if df_results['adra_ms'] is not None else None
        df_results['adsa_s'] = df_results['adsa_ms'] / 1000 if df_results['adsa_ms'] is not None else None
        df_results['total_delay_s'] = df_results['total_delay_ms'] / 1000 if df_results['total_delay_ms'] is not None else None
        
        print("--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---")
        print(df_results.to_string(float_format="%.2f"))
        print("\n" + "="*50 + "\n")

        print("--- Analisis Rata-rata Delay ---")
        
        # calculate average from adra_ms and adsa_ms
        average_adra = df_results['adra_ms'].mean()
        average_adra_s = average_adra / 1000 if average_adra is not None else None
        average_adsa = df_results['adsa_ms'].mean()
        average_adsa_s = average_adsa / 1000 if average_adsa is not None else None
        print(f"Rata-rata ADRA: {average_adra:.2f} ms ({average_adra_s:.2f} s)")
        print(f"Rata-rata ADSA: {average_adsa:.2f} ms ({average_adsa_s:.2f} s)")

        # Menghitung rata-rata keseluruhan dengan mengabaikan nilai None (inf)
        average_total_delay = df_results['total_delay_ms'].mean()
        print(f"Rata-rata Total Delay (ADRA + ADSA) untuk semua aplikasi: {average_total_delay:.2f} ms")
        
        # Aplikasi dengan delay terendah dan tertinggi
        app_min_delay = df_results['total_delay_ms'].idxmin()
        min_delay = df_results['total_delay_ms'].min()
        app_max_delay = df_results['total_delay_ms'].idxmax()
        max_delay = df_results['total_delay_ms'].max()
        
        print(f"Aplikasi dengan delay terendah: App '{app_min_delay}' ({min_delay:.2f} ms)")
        print(f"Aplikasi dengan delay tertinggi: App '{app_max_delay}' ({max_delay:.2f} ms)")


--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---
        adra_ms  adsa_ms  total_delay_ms  adra_s  adsa_s  total_delay_s
app_id                                                                 
0      80020.00 52011.00       132031.00   80.02   52.01         132.03
1      80020.00 60742.09       140762.09   80.02   60.74         140.76
2      40010.00 41342.33        81352.33   40.01   41.34          81.35
3      60015.00 34674.00        94689.00   60.02   34.67          94.69
4      60015.00 49154.57       109169.57   60.02   49.15         109.17
5      80020.00 34674.00       114694.00   80.02   34.67         114.69
6      60015.00 50678.67       110693.67   60.02   50.68         110.69
7      80020.00 42010.17       122030.17   80.02   42.01         122.03
8      40010.00 34674.00        74684.00   40.01   34.67          74.68
9      60015.00 52011.00       112026.00   60.02   52.01         112.03
10     60015.00 49345.22       109360.22   60.02   49.35         109.36
11     400

In [6]:
if __name__ == "__main__":
    # 1. Muat semua data dari file JSON
    network_def = load_json_file('exp_rev/networkDefinition.json')
    app_def = load_json_file('exp_rev/appDefinition.json')
    alloc_def = load_json_file('exp_rev/allocDefinition.json')

    if not all([network_def, app_def, alloc_def]):
        print("Satu atau lebih file definisi tidak dapat dimuat. Program berhenti.")
    else:
        # 2. Persiapkan struktur data
        network_graph = build_network_graph(network_def)
        placement_matrix = build_placement_matrix(alloc_def)
        
        # Siapkan daftar node untuk requester acak (kecuali node cloud ID 100)
        potential_requesters = [node['id'] for node in network_def.get('entity', []) if node.get('type') != 'CLOUD']
        if not potential_requesters:
            potential_requesters = [node for node in network_graph.nodes() if node != 100]

        # 3. Loop untuk semua aplikasi dan simpan hasilnya
        results_list = []
        PACKET_SIZE_BITS = 1_500_000  # Ukuran paket dalam bit (1.5 MB)

        for app_data in app_def:
            app_id = app_data['name']
            
            # Hitung ADSA
            adsa_result = calculate_adsa(app_data, placement_matrix, network_graph, PACKET_SIZE_BITS)
            
            # Hitung ADRA dengan requester acak
            entry_message = f"M.USER.APP.{app_id}"
            entry_module_name = next((t.get("module") for t in app_data.get("transmission", []) if t.get("message_in") == entry_message), None)
            
            adra_result = 0.0
            if entry_module_name and entry_module_name in placement_matrix and potential_requesters:
                entry_node = placement_matrix[entry_module_name][0]
                random_requester_id = random.choice(potential_requesters)
                adra_result = calculate_adra(random_requester_id, entry_node, network_graph, PACKET_SIZE_BITS)
            
            # Simpan hasil
            results_list.append({
                'app_id': app_id,
                'adra_ms': adra_result if adra_result != float('inf') else None,
                'adsa_ms': adsa_result,
                'total_delay_ms': (adra_result + adsa_result) if adra_result != float('inf') else None
            })

        # 4. Buat DataFrame dan lakukan analisis
        df_results = pd.DataFrame(results_list)
        df_results.set_index('app_id', inplace=True)

        # convert delay to seconds
        df_results['adra_s'] = df_results['adra_ms'] / 1000 if df_results['adra_ms'] is not None else None
        df_results['adsa_s'] = df_results['adsa_ms'] / 1000 if df_results['adsa_ms'] is not None else None
        df_results['total_delay_s'] = df_results['total_delay_ms'] / 1000 if df_results['total_delay_ms'] is not None else None
        
        print("--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---")
        print(df_results.to_string(float_format="%.2f"))
        print("\n" + "="*50 + "\n")

        print("--- Analisis Rata-rata Delay ---")
        
        # calculate average from adra_ms and adsa_ms
        average_adra = df_results['adra_ms'].mean()
        average_adra_s = average_adra / 1000 if average_adra is not None else None
        average_adsa = df_results['adsa_ms'].mean()
        average_adsa_s = average_adsa / 1000 if average_adsa is not None else None
        print(f"Rata-rata ADRA: {average_adra:.2f} ms ({average_adra_s:.2f} s)")
        print(f"Rata-rata ADSA: {average_adsa:.2f} ms ({average_adsa_s:.2f} s)")

        # Menghitung rata-rata keseluruhan dengan mengabaikan nilai None (inf)
        average_total_delay = df_results['total_delay_ms'].mean()
        print(f"Rata-rata Total Delay (ADRA + ADSA) untuk semua aplikasi: {average_total_delay:.2f} ms")
        
        # Aplikasi dengan delay terendah dan tertinggi
        app_min_delay = df_results['total_delay_ms'].idxmin()
        min_delay = df_results['total_delay_ms'].min()
        app_max_delay = df_results['total_delay_ms'].idxmax()
        max_delay = df_results['total_delay_ms'].max()
        
        print(f"Aplikasi dengan delay terendah: App '{app_min_delay}' ({min_delay:.2f} ms)")
        print(f"Aplikasi dengan delay tertinggi: App '{app_max_delay}' ({max_delay:.2f} ms)")

--- Rekapitulasi Hasil Delay untuk Semua Aplikasi ---
        adra_ms  adsa_ms  total_delay_ms  adra_s  adsa_s  total_delay_s
app_id                                                                 
0      80020.00 52011.00       132031.00   80.02   52.01         132.03
1      60015.00 60742.09       120757.09   60.02   60.74         120.76
2      60015.00 41342.33       101357.33   60.02   41.34         101.36
3      60015.00 34674.00        94689.00   60.02   34.67          94.69
4      40010.00 49154.57        89164.57   40.01   49.15          89.16
5      80020.00 34674.00       114694.00   80.02   34.67         114.69
6      40010.00 50678.67        90688.67   40.01   50.68          90.69
7      60015.00 42010.17       102025.17   60.02   42.01         102.03
8      60015.00 34674.00        94689.00   60.02   34.67          94.69
9      60015.00 52011.00       112026.00   60.02   52.01         112.03
10     60015.00 49345.22       109360.22   60.02   49.35         109.36
11     400

In [7]:
# based on appDefinition.json, allocDefinition.json, networkDefinition.json
# give me list of client id and entry point for each application

def load_app_entry_points(app_def: list, placement_matrix: dict) -> dict:
    """Mengambil daftar ID klien dan titik masuk untuk setiap aplikasi."""
    app_entry_points = {}
    for app in app_def:
        app_id = app['name']
        entry_message = f"M.USER.APP.{app_id}"
        entry_module_name = next((t.get("module") for t in app.get("transmission", []) if t.get("message_in") == entry_message), None)
        
        if entry_module_name and entry_module_name in placement_matrix:
            # Get all entry points for this module
            entry_nodes = placement_matrix[entry_module_name]
            app_entry_points[app_id] = entry_nodes
    return app_entry_points

# Get entry points for all applications
app_entry_points = load_app_entry_points(app_def, placement_matrix)

# Print in the requested format
for app_id, entry_points_list in app_entry_points.items():
    print(f"app: {app_id}, entry_point: {entry_points_list}")

# Create a summary DataFrame for better visualization
entry_points_summary = []
for app_id, entry_points_list in app_entry_points.items():
    entry_points_summary.append({
        'app_id': app_id,
        'entry_points': entry_points_list,
        'num_entry_points': len(entry_points_list)
    })

summary_df = pd.DataFrame(entry_points_summary)
summary_df.set_index('app_id', inplace=True)

print("\n--- Summary Table: Application Entry Points ---")
print(summary_df.to_string())

# Show statistics
print(f"\nTotal applications: {len(app_entry_points)}")
all_entry_points = [ep for ep_list in app_entry_points.values() for ep in ep_list]
print(f"Total entry points across all apps: {len(all_entry_points)}")
print(f"Unique entry points: {len(set(all_entry_points))}")

app: 0, entry_point: [36, 100]
app: 1, entry_point: [26, 69, 74, 84, 100]
app: 2, entry_point: [3, 100]
app: 3, entry_point: [7, 100]
app: 4, entry_point: [14, 55, 65, 71, 97, 98, 100]
app: 5, entry_point: [10, 100]
app: 6, entry_point: [6, 32, 46, 81, 83, 100]
app: 7, entry_point: [3, 4, 33, 55, 71, 100]
app: 8, entry_point: [1, 59, 100]
app: 9, entry_point: [68, 100]
app: 10, entry_point: [38, 40, 100]
app: 11, entry_point: [2, 27, 35, 70, 74, 100]
app: 12, entry_point: [12, 19, 45, 58, 100]
app: 13, entry_point: [15, 33, 35, 43, 67, 100]
app: 14, entry_point: [2, 4, 41, 52, 64, 77, 87, 96, 100]
app: 15, entry_point: [20, 24, 29, 34, 46, 100]
app: 16, entry_point: [61, 65, 88, 98, 100]
app: 17, entry_point: [8, 19, 42, 100]
app: 18, entry_point: [40, 61, 100]
app: 19, entry_point: [82, 100]

--- Summary Table: Application Entry Points ---
                               entry_points  num_entry_points
app_id                                                       
0                      

In [8]:
# Get unique entry point nodes across all applications
unique_entry_points = set()
for entry_points_list in app_entry_points.values():
    unique_entry_points.update(entry_points_list)

# Convert to sorted list for better readability
unique_entry_points_sorted = sorted(list(unique_entry_points))

print("--- Recap: Unique Entry Point Nodes ---")
print(f"Unique entry point nodes: {unique_entry_points_sorted}")
print(f"Total unique entry point nodes: {len(unique_entry_points_sorted)}")

# Create a frequency analysis to see which nodes are most commonly used as entry points
entry_point_frequency = {}
for entry_points_list in app_entry_points.values():
    for node in entry_points_list:
        entry_point_frequency[node] = entry_point_frequency.get(node, 0) + 1

# Sort by frequency (descending)
sorted_by_frequency = sorted(entry_point_frequency.items(), key=lambda x: x[1], reverse=True)

print("\n--- Entry Point Node Frequency Analysis ---")
print("Node ID | Frequency (Number of apps using this node)")
print("-" * 50)
for node_id, freq in sorted_by_frequency:
    print(f"{node_id:7} | {freq}")

# Identify cloud nodes vs edge nodes
cloud_nodes = [node for node in unique_entry_points_sorted if node == 100]  # Assuming 100 is cloud
edge_nodes = [node for node in unique_entry_points_sorted if node != 100]

print("\n--- Node Type Classification ---")
print(f"Cloud nodes serving as entry points: {cloud_nodes}")
print(f"Edge nodes serving as entry points: {edge_nodes}")
print(f"Number of edge nodes: {len(edge_nodes)}")
print(f"Number of cloud nodes: {len(cloud_nodes)}")

--- Recap: Unique Entry Point Nodes ---
Unique entry point nodes: [1, 2, 3, 4, 6, 7, 8, 10, 12, 14, 15, 19, 20, 24, 26, 27, 29, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43, 45, 46, 52, 55, 58, 59, 61, 64, 65, 67, 68, 69, 70, 71, 74, 77, 81, 82, 83, 84, 87, 88, 96, 97, 98, 100]
Total unique entry point nodes: 53

--- Entry Point Node Frequency Analysis ---
Node ID | Frequency (Number of apps using this node)
--------------------------------------------------
    100 | 20
     74 | 2
      3 | 2
     55 | 2
     65 | 2
     71 | 2
     98 | 2
     46 | 2
      4 | 2
     33 | 2
     40 | 2
      2 | 2
     35 | 2
     19 | 2
     61 | 2
     36 | 1
     26 | 1
     69 | 1
     84 | 1
      7 | 1
     14 | 1
     97 | 1
     10 | 1
      6 | 1
     32 | 1
     81 | 1
     83 | 1
      1 | 1
     59 | 1
     68 | 1
     38 | 1
     27 | 1
     70 | 1
     12 | 1
     45 | 1
     58 | 1
     15 | 1
     43 | 1
     67 | 1
     41 | 1
     52 | 1
     64 | 1
     77 | 1
     87 | 1
     96 | 1
  